In [167]:
import os
import pickle
import sklearn
print(sklearn.__version__)
import pandas
import numpy as np
import sys
from evaluation.metric import computeAccuracy, computeF1Score
from sklearn.metrics import roc_auc_score

0.17


In [168]:
clf_alc_initial = pickle.load(open('./alcohol classifiers/clf_alc_UPDATED.p', 'rb'))
# clf_fpa    = pickle.load(open('./alcohol classifiers/clf_fpa_UPDATED.p', 'rb'))
# clf_fpl_dl = pickle.load(open('./alcohol classifiers/clf_fpl_double_labeled', 'rb'))
df = pandas.read_csv("./data/alcohol_training_instances.csv")


In [169]:
clf_alc_initial.steps
# with open('structure.txt', 'w') as f:
#     tmp = ''.join(str(e) for e in clf_alc_initial.steps)
#     f.write(tmp)

[('features', FeatureUnion(n_jobs=1,
         transformer_list=[('text', Pipeline(steps=[('getter', ItemGetter(key='text')), ('tfidf', TfidfVectorizer(analyzer='char', binary=False, decode_error='strict',
          dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
          lowercase=True, max_df=1.0, max_features=71233, min_df=1,
     ...      token_pattern='(?u)\\b\\w\\w+\\b', tokenizer=None, use_idf=True,
          vocabulary=None))]))],
         transformer_weights=None)),
 ('scaler', Normalizer(copy=True, norm='l2')),
 ('clf',
  LogisticRegression(C=177.08103265690417, class_weight=None, dual=False,
            fit_intercept=True, intercept_scaling=1, max_iter=100,
            multi_class='ovr', n_jobs=None, penalty='l2', random_state=None,
            solver='liblinear', tol=0.000655077907893521, verbose=0,
            warm_start=None))]

Prepare the alcohol data

In [170]:
alc_labels = []
# extracting pre-labeled classification
for i in range(0,len(df.labels)):
    alc_labels.append(df.labels[i][12])
alc_labels = list(map(int, alc_labels))
print(alc_labels[:10])
print(len(alc_labels))
from sklearn.cross_validation import train_test_split
X_train, X_test, y_train, y_test = train_test_split(df, alc_labels, test_size=0.1, random_state=26)

[0, 1, 1, 1, 1, 1, 1, 1, 0, 1]
15651


First we check the initial model performance

In [171]:
clf_alc_initial.fit(X_train, y_train)
# predict in testing set.
y_alc_initial = clf_alc_initial.predict(X_test)
# print('length of testing set: ', len(y_alc_initial))
computeAccuracy(y_alc_initial, y_test)
auc_score_initial = roc_auc_score(y_test,y_alc_initial)
f1 = computeF1Score(y_test, y_alc_initial)
print('F1 Score: ', f1)
print('alcohol LR AUC: ', auc_score_initial)

D:\Anaconda3\envs\py36\lib\site-packages\sklearn\feature_extraction\text.py:1015: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  if hasattr(X, 'dtype') and np.issubdtype(X.dtype, np.float):


length of total comps:  1566
pretrained alcohol classifer:  261 different item in whole dataset
accuracy: 83.333333%
F1 Score:  0.8718703976435936
alcohol LR AUC:  0.8099516512591146


In [172]:
# %load_ext autoreload
%autoreload
from sklearn.linear_model import LogisticRegression
from sklearn.svm import LinearSVC
from scipy.stats import uniform
# %reload_ext pipelines.alcohol.AlcoholPipeline
from pipelines.alcohol import AlcoholPipeline
# import pipelines
# import importlib
# importlib.reload(pipelines)
# AlcoholPipeline = pipelines.alcohol.AlcoholPipeline
# from data import iterate_heirarchy

# should be uncomment in 0.17
# from classification.compute import CustomGridSearch

# from gridsearch import text_grid

# LR
clf_alc_LR = AlcoholPipeline(global_features=["text"]).pipeline(LogisticRegression())
clf_alc_LR.steps
# SVM
# clf_alc_SVM = AlcoholPipeline(global_features=["text"]).pipeline(LinearSVC())
# clf_alc_SVM

[('features', FeatureUnion(n_jobs=1,
         transformer_list=[('text', Pipeline(steps=[('getter', ItemGetter(key='text')), ('tfidf', TfidfVectorizer(analyzer='char', binary=False, decode_error='strict',
          dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
          lowercase=True, max_df=1.0, max_features=None, min_df=1,
      ...      token_pattern='(?u)\\b\\w\\w+\\b', tokenizer=None, use_idf=True,
          vocabulary=None))]))],
         transformer_weights=None)),
 ('scaler', Normalizer(copy=True, norm='l2')),
 ('clf',
  LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
            intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
            penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
            verbose=0, warm_start=False))]

In [173]:
clf_alc_LR.fit(X_train, y_train)
# pred_alc= clf_alc_LR.predict(df)
# print(pred_alc[:10])

D:\Anaconda3\envs\py36\lib\site-packages\sklearn\feature_extraction\text.py:1015: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  if hasattr(X, 'dtype') and np.issubdtype(X.dtype, np.float):


Pipeline(steps=[('features', FeatureUnion(n_jobs=1,
       transformer_list=[('text', Pipeline(steps=[('getter', ItemGetter(key='text')), ('tfidf', TfidfVectorizer(analyzer='char', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=T...ty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False))])

Predict in testing set

In [174]:
# predict in testing set.
y_alc_LR = clf_alc_LR.predict(X_test)
# print('length of testing set: ', len(y_alc_initial))
computeAccuracy(y_alc_LR, y_test)
auc_score_LR = roc_auc_score(y_test,y_alc_LR)
f1_score_LR = computeF1Score(y_test, y_alc_LR)
print('F1 Score: ', f1_score_LR)
print('alcohol LR AUC: ', auc_score_LR)

length of total comps:  1566
pretrained alcohol classifer:  473 different item in whole dataset
accuracy: 69.795658%
F1 Score:  0.7920879120879121
alcohol LR AUC:  0.6222367460929912


D:\Anaconda3\envs\py36\lib\site-packages\sklearn\feature_extraction\text.py:1015: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  if hasattr(X, 'dtype') and np.issubdtype(X.dtype, np.float):


Test in training set to figure out why the test accuracy is low

In [175]:
# test in training set to figure out why the test accuracy is low
y_train_alc_LR = clf_alc_LR.predict(X_train)
computeAccuracy(y_train_alc_LR, y_train)
auc_score_LR_train = roc_auc_score(y_train,y_train_alc_LR)
f1_score_LR_train = computeF1Score(y_train, y_train_alc_LR)
print('F1 Score: ', f1_score_LR_train)
print('alcohol LR AUC: ', auc_score_LR_train)

length of total comps:  14085
pretrained alcohol classifer:  3981 different item in whole dataset
accuracy: 71.735889%
F1 Score:  0.8098944654027984
alcohol LR AUC:  0.625503291223461


D:\Anaconda3\envs\py36\lib\site-packages\sklearn\feature_extraction\text.py:1015: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  if hasattr(X, 'dtype') and np.issubdtype(X.dtype, np.float):


Update structure parameters in LR model. To make sure they become same with the initial model

In [176]:
clf_alc_LR_updateParams = AlcoholPipeline(global_features=["text"]).pipeline(LogisticRegression())
params_LR = {'features__text__tfidf__dtype':np.int64, 'features__text__tfidf__max_features':71233, 'features__text__tfidf__ngram_range':(2,5) 
            }
# to see all parameters in model
# clf_alc_LR_updateParams.get_params().keys()
clf_alc_LR_updateParams.set_params(**params_LR)

Pipeline(steps=[('features', FeatureUnion(n_jobs=1,
       transformer_list=[('text', Pipeline(steps=[('getter', ItemGetter(key='text')), ('tfidf', TfidfVectorizer(analyzer='char', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=T...ty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False))])

Train the LR model with updated parameters, and predict in testing set

In [177]:
clf_alc_LR_updateParams.fit(X_train, y_train)
# predict in testing set.
y_alc_LR_updateParams = clf_alc_LR_updateParams.predict(X_test)
# print('length of testing set: ', len(y_alc_initial))
computeAccuracy(y_alc_LR_updateParams, y_test)
auc_score_LR_updateParams = roc_auc_score(y_test,y_alc_LR_updateParams)
f1_score_LR_updateParams = computeF1Score(y_test, y_alc_LR_updateParams)
print('F1 Score: ', f1_score_LR_updateParams)
print('alcohol LR AUC: ', auc_score_LR_updateParams)

D:\Anaconda3\envs\py36\lib\site-packages\sklearn\feature_extraction\text.py:1015: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  if hasattr(X, 'dtype') and np.issubdtype(X.dtype, np.float):


length of total comps:  1566
pretrained alcohol classifer:  227 different item in whole dataset
accuracy: 85.504470%
F1 Score:  0.8927727916863486
alcohol LR AUC:  0.8189652004556647


We can see the model we build with updated parameters achieve same results as the initial model, or maybe even better.

In [178]:
pred_alc= clf_alc_SVM.predict(df)
print(pred_alc[:10])

NotFittedError: TfidfVectorizer - Vocabulary wasn't fitted.

In [ ]:
# Wrong way to update classifier
# from sklearn.svm import LinearSVC
# from sklearn.pipeline import FeatureUnion
# clf_alc_initial.named_steps['clf'] = LinearSVC()
# clf_alc_initial.named_steps['clf']